In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)

fname = 'OMCD_1'
log_file = fname + '.txt'

with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')


df = pd.read_csv('datasets/OMCD/train.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3
df_test = pd.read_csv('datasets/OMCD/test.csv', encoding='utf-8', engine='python') #, quotechar="'"  , quoting=3


      
display(df.columns)
display(df_test.columns)
display(df[:4])



classes = set(df['off'].values)
display(classes)

df['off'] = df['off'].astype('category')
df['label'] = df['off'].cat.codes


df_test['off'] = df_test['off'].astype('category')
df_test['label'] = df_test['off'].cat.codes

df = df[['comment', 'label']]
df_test = df_test[['comment', 'label']]
classes_num = len(classes)
display(classes_num)
display(len(df))
display(len(df_test))


ds_t = Dataset.from_pandas(df)
ds_v = Dataset.from_pandas(df_test)

display(ds_t)
display(ds_v)

max_sequence_length = 128



models = [ 
        'aubmindlab/bert-base-arabertv02-twitter',
        'CAMeL-Lab/bert-base-arabic-camelbert-da',
        'qarib/bert-base-qarib', 
        'UBC-NLP/MARBERT',
        'UBC-NLP/MARBERTv2',
]





for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds_t
        dataset_validation = ds_v                                                     
        
      

        def preprocess_function(examples):
            return tokenizer(examples['comment'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 20
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 64
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 60, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 60
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv(f'{fname}.csv')
display(best_results)



Index(['Unnamed: 0', 'comment', 'off'], dtype='object')

Index(['Unnamed: 0', 'comment', 'off'], dtype='object')

,Unnamed: 0,comment,off
0,2908,فنانين الكبت والفساد .عقلية جنسية لا غير. العفن وليس الفن. شعب فيه اغلبية مستعدة للجنس وعندها قابلية .,1
1,1055,الدعارة هربت منها في المحمدية و سكنت في بوزنيقة و هي بحال بحال موجودة في كل المدن و السبب ملكنا زامل و الناس فقراء بالزاف,1
2,181,كون غير خريتي و مدرتيش هادشي,1
3,4313,لا حول ولا قوة الا بالله العلي العظيم لا حول ولا قوة الا بالله العلي العظيم. استغفرالله العظيم واتوب اليه. مساكن الوالدين هما اللي في الواجهة. شعب لا يرحم,0


{0, 1}

2

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERTv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6419 [00:00<?, ? examples/s]

Map:   0%|          | 0/1605 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
60,0.484500,0.360196,0.841745,0.839471
120,0.341500,0.449397,0.847352,0.845896
180,0.230300,0.401768,0.854206,0.853072
240,0.151800,0.539415,0.831776,0.831606
300,0.111800,0.492254,0.850467,0.848710
360,0.069100,0.621607,0.852960,0.852174
420,0.081700,0.770473,0.833022,0.827018
480,0.063600,0.708961,0.857321,0.856289
540,0.053100,0.646465,0.849221,0.848115
600,0.045400,0.832403,0.846729,0.844527


,Model,Accuracy,F1
0,CAMeL-Lab/bert-base-arabic-camelbert-da,0.832399,0.829914
3,UBC-NLP/MARBERT,0.842368,0.839179
6,UBC-NLP/MARBERTv2,0.861682,0.860380
9,aubmindlab/bert-base-arabertv02-twitter,0.859190,0.857723
10,qarib/bert-base-qarib,0.846106,0.845090
